# League of Data - Model Evaluation

En el Notebook anterior, **League of Data - Data Prepropocessing** se realizó todo el proceso de limipieza de la información y se generó un conjunto de datos basado en cocientes para poder comparar dos equipos y así caracterizar una partida.

En este Notebook nos enfocaremos en analizar la estructura del conjunto de datos y encontrar por medio de este, el modelo que mejor prediga cual es el equipo que resultará victorioso en una partida.

Para alcanzar este objetivo, se plantearon los siguientes pasos.

* Visualización del estado de los datos.
* Entrenamiento de modelos.
* Analísis de características.
* Selección y extraccion de variables.

### VISUALIZACIÓN DEL ESTADO DE LOS DATOS
